# Proyecto

En este modulo vamos a hacer un proyecto entero de aprendizaje supervisado, pasando por todos los pasos del mismo, los cuales son:

* Definición del problema.
* Búsqueda de datos. 
* Análisis exploratorio y limpieza.
* Separación de entrenamiento y validación.
* Entrenamiento.
* Prueba.
* Presentación de resultados. 

## Definición del problema

Todo proyecto de aprendizaje automático parte de una pregunta que se quiere responder, en este caso la pregunta es:

**¿Cuál será la temperatura de acuerdo a determinadas características?**

## Búsqueda de datos

El dataset que se utilizará es sobre datos climáticos donde de acuerdo a determinadas variables (humedad, velocidad del viento, rumbo del viento, visibilidad, presión, si llueve y descripción) se puede predecir la temperatura.
Esta basado en el dataset descargable en [Kaggle](https://www.kaggle.com/zakriarehman/weather-data-for-linear-regression) pero tiene realizadas algunas modificaciones por lo que debe ser tomado desde el archivo csv descargado desde la plataforma. 


#### Exploración del dataset

En el transcurso de este Módulo vamos a hacer un proyecto de Ciencia de Datos desde el principio viendo en cada encuentro un aspecto específico del mismo: exploración inicial y visualizaciones, limpieza de datos (nulos, variables dummies), división de datos para entrenamiento y testeo y entrenamiento y testeo de un modelo.



Comenzemos por leer el dataset y hacer la primer exploración del mismo. 

In [ ]:
#importamos las librerias que utilizaremos

import pandas as pd
import matplotlib.pyplot as plt  
import seaborn as sns

#### Desde el Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
data = pd.read_csv("Dataset Clima.csv")
data.head()

,id,Humidity,Wind_Speed_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
0,0,0.92,11.27,130.0,8.05,1021.60,no,Cold,-0.56
1,1,0.73,20.93,330.0,16.10,1017.00,si,Warm,21.11
2,2,0.97,5.97,193.0,14.91,1013.99,si,Normal,16.60
3,3,0.82,3.22,300.0,16.10,1031.59,si,Cold,1.60
4,4,0.60,10.88,116.0,9.98,1020.88,si,Cold,2.19
...,...,...,...,...,...,...,...,...,...
95,95,0.46,11.45,10.0,11.32,1007.18,si,Warm,19.83
96,96,0.64,16.31,8.0,9.98,1023.60,si,Normal,15.83
97,97,0.85,10.63,136.0,12.12,1018.12,si,Normal,17.57
98,98,0.85,5.06,87.0,15.83,1015.64,si,Warm,22.55


#### Desde el archivo descargado en la computadora

In [ ]:
from google.colab import files
import io

#filesUploaded = files.upload()

In [ ]:
#vemos los primeros registros del dataset

data.head(3)

In [ ]:
# Vemos el tamaño del dataset

data.shape

(10000, 9)

In [ ]:
# Vemos los tipos de datos

data.dtypes

id                       int64
Humidity               float64
Wind_Speed_kmh         float64
rumbo_viento_grados    float64
visibilidad_km         float64
presion_mbar           float64
lluvia                  object
descripcion             object
temperatura            float64
dtype: object

In [ ]:
data['descripcion'].isna().sum()
data[data['descripcion'].isna()]

,id,Humidity,Wind_Speed_kmh,rumbo_viento_grados,visibilidad_km,presion_mbar,lluvia,descripcion,temperatura
549,549,0.55,20.46,160.0,10.21,1020.71,si,NaN,8.91
1485,1485,0.56,16.04,33.0,10.35,1021.43,si,NaN,2.83
2224,2224,0.48,11.27,120.0,9.98,1011.10,si,NaN,16.11
3485,3485,0.45,13.78,341.0,16.10,1019.04,si,NaN,23.79
5382,5382,0.92,4.73,170.0,3.53,1006.13,no,NaN,-5.16
6582,6582,0.65,27.10,319.0,9.98,0.00,si,NaN,4.95
7815,7815,NaN,NaN,NaN,NaN,NaN,si,NaN,0.98
9614,9614,0.82,11.27,10.0,16.10,1019.90,si,NaN,0.56


Es importante observar si el Dataset tiene datos nulos, es decir registros vacíos que no tienen ningún valor. Para esto utilizaremos el método *isnull()* pero veremos esta cuestión en profundidad en el encuentro que viene.

In [ ]:
data.isnull().sum()

id                     0
Humidity               5
Wind_Speed_kmh         5
rumbo_viento_grados    5
visibilidad_km         5
presion_mbar           9
lluvia                 1
descripcion            8
temperatura            0
dtype: int64